<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1397e29577a4cab1bc3708331beea12dc9a7b2e2b07201878ff36dd820df8fc7
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-17 12:20:37
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.46 C
-------------------
Today PnL: 29.37 K (0.2%)
Current PnL: -19.91 L (-13.05%)
CY Booked + Current PnL: -6.39 L (-4.19%)
-------------------
Total profit:  2.04 L
Total loss:  -21.95 L
-------------------
Total Booked + Current PnL: 20.59 L (16.39%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.29%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 90.79 L (62.4%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.5%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,101.49,51.0,M-SC,3.42,86853.0,-13924.0,14018.0,-0.08,-13.82,16.14,0.09,253.0,-0.99,0.60,14.11,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,143.30,81.0,M-SC,0.80,159576.0,-14983.0,15016.0,-0.92,-8.58,9.41,0.02,234.0,-1.00,1.10,38.80,OX40N,NTT,PAINTS
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.56,88605.0,-2073.0,16977.0,0.66,-2.29,19.16,16.44,63.0,-0.12,0.61,17.43,X40,ATH,PHARMA
5,ANGELONE,3033.00,63.47,77.0,X-SC,7.38,229682.0,38676.0,19018.0,2.03,20.25,8.28,30.21,97.0,2.03,1.58,45.36,X40N,NTT,FINANCE
35,ICICIGI,2252.93,-15.31,57.0,X-MC,7.50,188585.0,14265.0,20933.0,1.13,8.18,11.10,20.19,68.0,0.68,1.30,24.53,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.0,66.67,57.0,H-SC,13.77,153683.0,-25388.0,91119.0,9.55,-14.18,59.29,36.71,167.0,-0.28,1.06,52.48,XR,NTT,BANKS
66,SIEMENS,4671.5,-2.45,58.0,H-LC,1.56,158800.0,-27295.0,74779.0,2.98,-14.67,47.09,25.51,49.0,-0.37,1.09,17.02,AR,ATH,ELECTRICAL
55,QUESS,986.0,-52.07,35.0,X-SC,37.20,47499.0,-17507.0,168436.0,2.38,-26.93,354.61,232.18,83.0,-0.10,0.33,2.38,XY24,NTT,MISC
22,DEN,75.0,86.53,43.0,M-SC,5.96,52064.0,-32400.0,67938.0,2.36,-38.36,130.49,42.07,235.0,-0.48,0.36,10.57,AR,NTT,ENTERTAINMENT
75,TITAGARH,1548.0,10.99,52.0,H-SC,4.48,201655.0,-40522.0,151282.0,2.08,-16.73,75.02,45.74,174.0,-0.27,1.39,31.16,XY24,NTT,ENGINEERING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.0,-20.63,26.0,X-LC,0.29,245036.0,-31374.0,452532.0,-4.37,-11.35,184.68,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
24,EASEMYTRIP,26.4,-21.71,33.0,M-SC,20.26,75429.0,-101386.0,182184.0,-2.77,-57.34,241.53,45.70,205.0,-0.56,0.52,0.00,XY24,NTT,TRAVEL
19,COFFEEDAY,80.0,-51.71,33.0,L-SC,21.19,71808.0,-41741.0,81789.0,-2.60,-36.76,113.90,35.27,270.0,-0.51,0.49,75.75,XR,NTT,HOTELS
14,BSOFT,831.7,-11.48,56.0,H-SC,8.63,97002.0,-43651.0,115083.0,-2.14,-31.03,118.64,50.79,151.0,-0.38,0.67,12.94,XR,ATH,IT
28,GREENPANEL,537.0,191.34,35.0,M-SC,5.38,130101.0,-52977.0,131961.0,-1.99,-28.94,101.43,43.14,240.0,-0.40,0.89,22.38,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.52,49.0,M-SC,10.20,148188.0,544.0,158783.0,1.20,0.37,107.15,107.91,216.0,0.00,1.02,3.38,OX40N,ATH,CABLES
69,STARHEALTH,761.00,37.49,71.0,H-SC,15.52,283424.0,2301.0,130573.0,-0.28,0.82,46.07,47.26,171.0,0.02,1.95,51.85,XY24,NTT,INSURANCE
85,WIPRO,420.00,-14.25,51.0,M-LC,5.59,150902.0,-43.0,109494.0,-0.40,-0.03,72.56,72.51,99.0,-0.00,1.04,5.99,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.52,49.0,M-SC,10.20,148188.0,544.0,158783.0,1.20,0.37,107.15,107.91,216.0,0.00,1.02,3.38,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,143.30,81.0,M-SC,0.80,159576.0,-14983.0,15016.0,-0.92,-8.58,9.41,0.02,234.0,-1.00,1.10,38.80,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.49,51.0,M-SC,3.42,86853.0,-13924.0,14018.0,-0.08,-13.82,16.14,0.09,253.0,-0.99,0.60,14.11,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.98,49.0,H-SC,2.37,223146.0,-46521.0,82854.0,-0.35,-17.25,37.13,13.47,158.0,-0.56,1.53,13.46,XY24,NTT,PAINTS
67,SIS,528.00,2017.10,46.0,H-SC,2.37,85438.0,-25594.0,49204.0,-0.34,-23.05,57.59,21.26,166.0,-0.52,0.59,15.43,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.0,63.47,77.0,X-SC,7.38,229682.0,38676.0,19018.0,2.03,20.25,8.28,30.21,97.0,2.03,1.58,45.36,X40N,NTT,FINANCE


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,37.49,71.0,H-SC,15.52,283424.0,2301.0,130573.0,-0.28,0.82,46.07,47.26,171.0,0.02,1.95,51.85,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,19.75,57.0,M-SC,36.94,161811.0,12591.0,131585.0,2.07,8.44,81.32,96.62,245.0,0.10,1.11,75.11,XY25,NTT,FINANCE
25,FINCABLES,1641.55,-18.52,49.0,M-SC,10.20,148188.0,544.0,158783.0,1.20,0.37,107.15,107.91,216.0,0.00,1.02,3.38,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-47.58,54.0,H-SC,4.44,130815.0,7479.0,124143.0,0.69,6.06,94.90,106.72,139.0,0.06,0.90,30.86,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.58,54.0,H-SC,4.44,130815.0,7479.0,124143.0,0.69,6.06,94.90,106.72,139.0,0.06,0.90,30.86,AR,ATH,MISC
25,FINCABLES,1641.55,-18.52,49.0,M-SC,10.20,148188.0,544.0,158783.0,1.20,0.37,107.15,107.91,216.0,0.00,1.02,3.38,OX40N,ATH,CABLES
85,WIPRO,420.00,-14.25,51.0,M-LC,5.59,150902.0,-43.0,109494.0,-0.40,-0.03,72.56,72.51,99.0,-0.00,1.04,5.99,XR,NTT,IT
84,WHIRLPOOL,2270.00,-42.22,38.0,M-SC,8.85,113197.0,-1601.0,95640.0,-1.17,-1.39,84.49,81.92,236.0,-0.02,0.78,34.01,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,-14.97,35.0,H-MC,5.59,207143.0,-2431.0,77099.0,-0.76,-1.16,37.22,35.63,119.0,-0.03,1.42,15.32,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,10.61,24.0,X-SC,9.77,84361.0,-44309.0,89608.0,-1.04,-34.44,106.22,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,1065.00,-20.63,26.0,X-LC,0.29,245036.0,-31374.0,452532.0,-4.37,-11.35,184.68,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,-19.71,29.0,X-MC,5.70,187920.0,-12427.0,42921.0,-0.89,-6.20,22.84,15.23,66.0,-0.29,1.29,24.05,X40N,NTT,REALTY
27,GLAXO,3466.20,-19.47,31.0,X-MC,6.39,199792.0,508.0,77499.0,0.73,0.25,38.79,39.15,60.0,0.01,1.37,29.33,X40,ATH,PHARMA
26,GILLETTE,11206.78,-17.17,32.0,X-SC,5.89,256020.0,2124.0,80185.0,-0.06,0.84,31.32,32.42,85.0,0.03,1.76,16.06,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-20.63,26.0,X-LC,0.29,245036.0,-31374.0,452532.0,-4.37,-11.35,184.68,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.74,47.0,X-LC,0.91,199121.0,-1017.0,21903.0,-0.23,-0.51,11.00,10.44,5.0,-0.05,1.37,5.03,X40,NTT,FMCG
34,HONAUT,58357.33,-25.10,35.0,X-SC,1.00,106395.0,-21543.0,68678.0,-0.18,-16.84,64.55,36.84,90.0,-0.31,0.73,8.92,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.46,64.0,X-MC,1.17,183872.0,7134.0,47604.0,-0.30,4.04,25.89,30.97,75.0,0.15,1.26,17.10,X40,ATH,INSURANCE
66,SIEMENS,4671.50,-2.45,58.0,H-LC,1.56,158800.0,-27295.0,74779.0,2.98,-14.67,47.09,25.51,49.0,-0.37,1.09,17.02,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,10.61,24.0,X-SC,9.77,84361.0,-44309.0,89608.0,-1.04,-34.44,106.22,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,1065.00,-20.63,26.0,X-LC,0.29,245036.0,-31374.0,452532.0,-4.37,-11.35,184.68,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
53,PAGEIND,51769.93,-28.33,39.0,X-MC,7.74,159360.0,-4380.0,47712.0,0.19,-2.67,29.94,26.47,55.0,-0.09,1.10,0.97,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-18.58,35.0,X-MC,6.63,175972.0,-24534.0,121685.0,-0.22,-12.24,69.15,48.45,64.0,-0.20,1.21,1.48,X40N,ATH,FINANCE
54,PGHH,17907.65,-31.57,35.0,X-MC,2.08,194760.0,-6060.0,73853.0,-0.40,-3.02,37.92,33.76,57.0,-0.08,1.34,2.02,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-52.07,35.0,X-SC,37.20,47499.0,-17507.0,168436.0,2.38,-26.93,354.61,232.18,83.0,-0.10,0.33,2.38,XY24,NTT,MISC
59,RELAXO,1176.00,-40.95,41.0,X-SC,3.23,80445.0,-64715.0,140642.0,0.41,-44.58,174.83,52.31,91.0,-0.46,0.55,7.68,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,10.61,24.0,X-SC,9.77,84361.0,-44309.0,89608.0,-1.04,-34.44,106.22,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.56,88605.0,-2073.0,16977.0,0.66,-2.29,19.16,16.44,63.0,-0.12,0.61,17.43,X40,ATH,PHARMA
34,HONAUT,58357.33,-25.10,35.0,X-SC,1.00,106395.0,-21543.0,68678.0,-0.18,-16.84,64.55,36.84,90.0,-0.31,0.73,8.92,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.54,59.0,X-LC,8.48,291560.0,-54396.0,121085.0,-0.14,-15.72,41.53,19.28,1.0,-0.45,2.00,7.79,X40,ATH,IT
41,INFY,2275.00,-20.67,52.0,X-LC,4.01,319180.0,6292.0,165399.0,-0.29,2.01,51.82,54.87,2.0,0.04,2.19,10.46,X40,BTT,IT
76,TMPV,1065.00,-20.63,26.0,X-LC,0.29,245036.0,-31374.0,452532.0,-4.37,-11.35,184.68,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
82,VBL,671.64,-17.64,49.0,X-LC,5.98,295107.0,-20735.0,133388.0,0.69,-6.56,45.20,35.67,4.0,-0.16,2.03,6.38,X40N,ATH,FMCG
43,ITC,452.00,-40.74,47.0,X-LC,0.91,199121.0,-1017.0,21903.0,-0.23,-0.51,11.00,10.44,5.0,-0.05,1.37,5.03,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-51.71,33.0,L-SC,21.19,71808.0,-41741.0,81789.0,-2.60,-36.76,113.90,35.27,270.0,-0.51,0.49,75.75,XR,NTT,HOTELS
7,ASIANTILES,137.00,7605.56,64.0,L-SC,17.12,84517.0,-9293.0,85912.0,-0.32,-9.91,101.65,81.67,271.0,-0.11,0.58,63.67,XR,NTT,CERAMICS
51,MASFIN,398.61,-18.25,53.0,H-SC,7.48,93000.0,-4980.0,26579.0,0.24,-5.08,28.58,22.05,164.0,-0.19,0.64,35.18,XR,ATH,FINANCE
38,INDIAMART,4810.62,-47.58,54.0,H-SC,4.44,130815.0,7479.0,124143.0,0.69,6.06,94.90,106.72,139.0,0.06,0.90,30.86,AR,ATH,MISC
80,VAIBHAVGBL,521.00,72.45,61.0,H-SC,9.45,149714.0,-33061.0,144130.0,0.13,-18.09,96.27,60.77,159.0,-0.23,1.03,34.83,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,143.30,81.0,M-SC,0.80,159576.0,-14983.0,15016.0,-0.92,-8.58,9.41,0.02,234.0,-1.00,1.10,38.80,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,72.45,61.0,H-SC,9.45,149714.0,-33061.0,144130.0,0.13,-18.09,96.27,60.77,159.0,-0.23,1.03,34.83,XR,NTT,JEWELLERY
6,ASIANPAINT,3465.66,0.35,85.0,X-LC,9.53,255156.0,3388.0,49832.0,-0.24,1.35,19.53,21.13,31.0,0.07,1.75,37.25,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-699.86,71.0,L-MC,6.16,48592.0,-28614.0,54005.0,0.50,-37.06,111.14,32.89,259.0,-0.53,0.33,33.82,XR,NTT,BANKS
14,BSOFT,831.70,-11.48,56.0,H-SC,8.63,97002.0,-43651.0,115083.0,-2.14,-31.03,118.64,50.79,151.0,-0.38,0.67,12.94,XR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.59
1,25,44.44
2,50,75.69


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.63
MC    27.73
LC    24.61
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.35
X40      20.49
X40N     12.27
XY25     10.08
XR        9.62
AR        9.00
OX40N     7.54
X200      1.80
SR        1.01
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.34
X-MC    21.37
X-LC    19.69
M-SC    13.27
X-SC     8.60
H-MC     4.66
H-LC     2.89
L-SC     1.42
M-MC     1.37
M-LC     1.04
L-LC     0.99
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.64,-4.00,37.63
IT,12.45,-16.14,78.22
FINANCE,11.97,-8.52,57.71
MISC,7.51,-18.80,78.73
PAINTS,5.99,-5.79,21.56
ELECTRICAL,5.59,-10.99,50.33
INSURANCE,4.51,3.61,30.36
PHARMA,3.40,-0.81,34.62
AUTO,3.36,-17.35,129.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3326771.0,20
AR,1260463.0,10
XR,1255580.0,13
X40,987688.0,14
OX40N,695773.0,10
X40N,682875.0,9
XY25,467474.0,7
SR,274116.0,2
MH,66299.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3463246.0,25
M-SC,1420012.0,16
X-LC,1201417.0,11
X-MC,1133276.0,15
X-SC,824573.0,8
H-MC,399900.0,3
L-SC,254303.0,3
H-LC,136745.0,2
M-LC,109494.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1163589.0      6
           AR         842400.0      5
           XR         780696.0      7
M-SC       XY24       747538.0      6
X-LC       XY24       516035.0      2
           X40        408970.0      5
X-MC       X40        408925.0      7
X-SC       XY24       374541.0      2
X-MC       XY24       344272.0      3
H-SC       OX40N      336146.0      4
X-SC       X40N       280239.0      4
H-SC       SR         274116.0      2
M-SC       OX40N      273025.0      5
H-MC       AR         219104.0      2
X-MC       X40N       201831.0      3
X-LC       X40N       200805.0      2
H-MC       XY24       180796.0      1
X-MC       XY25       178248.0      2
X-SC       X40        169793.0      2
L-SC       XR         167701.0      2
M-SC       XR         143684.0      2
           XY25       131585.0      1
           AR         124180.0      2
M-LC       XR         109494.0      1
L-SC       OX40N       86602.0      1
X-LC       XY25        75607.0      2
H-LC       AR          74779.0      1
H-SC       MH          66299.0      1
H-LC       X200        61966.0      1
L-MC       XR          54005.0      1
M-MC       XY25        46385.0      1
L-LC       XY25        35649.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
